In [1]:
import pyspark
from sklearn import preprocessing
import xgboost as xgb
import pandas as pd
import numpy as np
sc = pyspark.SparkContext('local[*]')

In [2]:
postulantes = pd.read_csv('Sources/postulantes_proc.csv').drop('Unnamed: 0', axis=1)
avisos = pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv').drop('Unnamed: 0', axis=1)
postulaciones = pd.read_csv('Sources/postulaciones_for_training.csv').drop('Unnamed: 0', axis=1)

In [3]:
postulaciones = postulaciones.merge(avisos, on='idaviso')
postulaciones = postulaciones.merge(postulantes, on='idpostulante')

In [4]:
postulaciones.drop(['titulo','descripcion'],axis=1, inplace=True)

In [6]:
test = pd.read_csv("Sources/test_final_100k.csv")

In [7]:
test = test.merge(avisos, on='idaviso')
test = test.merge(postulantes, on='idpostulante')
test['sepostulo']=1
test.drop(['titulo','descripcion'],axis=1, inplace=True)
test.head()

,id,idaviso,idpostulante,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo,sepostulo
0,0,739260,6M9ZQR,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,17,3.0,0.0,0.0,0.0,0.0,0.0,3.0,1976-03-29,FEM,1
1,1,739260,6v1xdL,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,17,0.0,0.0,3.0,0.0,0.0,3.0,0.0,1987-06-27,MASC,1
2,2,739260,ezRKm9,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,17,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1982-02-17,FEM,1
3,3,758580,1Q35ej,Full-time,Otro,Tecnologia / Sistemas,TeleTech,11,0.0,0.0,0.0,3.0,0.0,0.0,3.0,1949-12-24,MASC,1
4,4,758580,EAN4J6,Full-time,Otro,Tecnologia / Sistemas,TeleTech,11,0.0,0.0,0.0,0.0,0.0,2.0,2.0,1986-05-27,FEM,1


In [8]:
def dates_format(date):
    #print(date)
    try:
        date_new = date.split('-')
        date_new = int("".join(date_new))
        return date_new
    except AttributeError:
        return date

In [9]:
fechasRDD = sc.parallelize(postulaciones['fechanacimiento'])
postulaciones['fechanacimiento'] = fechasRDD.map(lambda x: dates_format(x)).collect()
fechasRDD = sc.parallelize(test['fechanacimiento'])
test['fechanacimiento'] = fechasRDD.map(lambda x: dates_format(x)).collect()
fechasRDD = None

In [10]:
resultado = postulaciones['sepostulo']
resultadob = test['sepostulo']
postulaciones.drop(['sepostulo','idaviso','idpostulante'],axis=1,inplace=True)
test.drop(['sepostulo','idaviso','idpostulante'],axis=1,inplace=True)
test.drop('id',axis=1,inplace=True)

In [12]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in postulaciones.columns:
    if not postulaciones[col].dtype == 'O': continue
    postulaciones[col] = postulaciones.apply(lambda x: str(x[col]),axis=1)
    test[col] = test.apply(lambda x: str(x[col]),axis=1)
    le.fit(pd.concat([postulaciones[col],test[col]]))
    test[col] = le.transform(test[col])
    postulaciones[col] = le.transform(postulaciones[col])
postulaciones.head()

,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo
0,1,4,120,2428,8,0.0,0.0,0.0,0.0,0.0,0.0,3.0,19810602.0,1
1,1,4,120,2428,8,0.0,0.0,0.0,0.0,0.0,3.0,2.0,19911011.0,2
2,1,2,36,3236,2,0.0,0.0,0.0,0.0,0.0,3.0,2.0,19911011.0,2
3,1,4,36,2228,8,0.0,0.0,0.0,0.0,0.0,3.0,2.0,19911011.0,2
4,1,4,64,154,14,0.0,0.0,0.0,0.0,0.0,3.0,2.0,19911011.0,2


In [13]:
postulaciones.rename(columns={x:int(y) for x,y in zip(postulaciones.columns,range(1,len(postulaciones.columns)+1))},inplace=True)
test.rename(columns={x:int(y) for x,y in zip(test.columns,range(1,len(test.columns)+1))},inplace=True)

In [14]:
aviso = None
postulantes = None